# Using a Transferred DCNN (VGG16). 

*   Training data from 5 subjects
*   Test data from remaining subject
*   0 degree aspect angle

*   Take a DCNN trained on image classification
*   Adapt to spectrogram classification
  




Inspired by "Micro-Doppler Based Classification of Human Aquatic Activities via Transfer Learning of Convolutional Neural Networks"

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [0]:
# Needed to allow editing using PyCharm
%load_ext autoreload
%autoreload 2

Needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [2]:
import os
if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif os.getcwd() == 'D:\\Google Drive\\Level-4-Project\\notebooks\\transferred_DCNN_experiments':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
else:
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/cifar_initialised'
MODEL_PATH = BASE_PATH + 'models/transferred_DCNN/'
FIGURE_PATH = BASE_PATH + 'reports/figures/transferred_DCNN/'
REPORT_PATH = BASE_PATH + 'reports/transferred_DCNN/'
    
from src.models.original_models import cnn_various_depths
from src.data import load_data
from src.visualization import visualize, multiple_plots

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Running setup.py develop for src


Using TensorFlow backend.


Import remaining packages

In [0]:
import numpy as np
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from sklearn.model_selection import StratifiedKFold

In [0]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

## Experiment Setup and Parameter Definition

### Experiment Togggles

In [0]:
load_models = False  # if True, load saved models instead of retraining
train_model_using_k_fold = True
save_graph = False
save_models = False
save_reports = False

### Parameter Definition

In [0]:
batch_size = 100
nb_classes = 7
nb_epoch = 3
nb_train_samples = 35595
n_folds = 3 # number of folds for k-fold cross validation

# input image dimensions (images are spectrograms)
img_rows, img_cols = 75, 75

### Data Loading

In [0]:
# the data, shuffled and split between train and test sets
(data_train, labels_train), (data_test, labels_test) = load_data.load_data(DATA_PATH, nb_train_samples)

In [0]:
target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']

Stack three times to fit VGG16 (RGB) treated as grayscale

In [0]:
data_train = np.hstack((data_train, data_train, data_train))
data_test = np.hstack((data_test, data_test, data_test))

In [0]:
def make_model(img_rows, img_cols, nb_classes):
    model = VGG16(weights = "imagenet", include_top=False, input_shape=(3, img_rows, img_cols))
    x = model.output
    x = Flatten()(x)
    # x = Dense(512, activation="relu")(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    # x = Dense(512, activation="relu")(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(nb_classes, activation="softmax")(x)
    model = Model(input = model.input, output = predictions)
    sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])
    return model

In [0]:
# set type and normalize the data
def process_train_test_data(data_train, data_test):
    data_train = data_train.astype('float32')
    data_test = data_test.astype('float32')
    data_train /= 255
    data_test /= 255
    return data_train, data_test

In [0]:
# convert class vectors to binary class matrices
def process_train_test_labels(labels_train, labels_test, nb_classes):
    labels_train = np_utils.to_categorical(labels_train, nb_classes)
    labels_test = np_utils.to_categorical(labels_test, nb_classes)
    return labels_train, labels_test

## Training and Evaluating Models

In [0]:
def train_and_evaluate_model(model, data_train, labels_train, data_test, labels_test, target_names, nb_classes):
    labels_train, labels_test = process_train_test_labels(labels_train, labels_test, nb_classes)
    data_train, data_test = process_train_test_data(data_train, data_test)
    
    history = model.fit(
        data_train,
        labels_train,
        batch_size=batch_size,
        epochs=nb_epoch,
        validation_data=(data_test, labels_test),
        shuffle=True, 
        verbose=1)
    # make prediction on test set
    y_pred = np.argmax(model.predict(data_test),axis=1)
    classification_rep = classification_report(np.argmax(labels_test,axis=1), y_pred, target_names=target_names)
    confusion = confusion_matrix(np.argmax(labels_test,axis=1), y_pred)  
    # return the results
    return {
        "history":history,
        "classification_report":classification_rep,
        "confusion_matrix":confusion,
        "model": model
    }

### K-Fold Cross Validation

In [27]:
if train_model_using_k_fold:
    # combine train and test sets as k-folds splits these instead
    data_train = np.vstack((data_train, data_test))
    labels_train = np.vstack((labels_train, labels_test))
    skf = StratifiedKFold(n_splits=n_folds, random_state=None, shuffle=True)
    fold_num = 1
    results = {}
    for train_index, test_index in skf.split(X_train, y_train):
        print ("Running Fold", fold_num, "/", n_folds)
        model = None # Clearing the NN.
        model = make_model(img_rows, img_cols, nb_classes)
        fold_results = train_and_evaluate_model(
            model,
            data_train[train_index],
            labels_train[train_index],
            data_train[test_index],
            labels_train[test_index],
            target_names,
            nb_classes)
        results[fold_num] = fold_results
        fold_num+=1

(42714, 3, 75, 75) (42714, 1)
Running Fold 1 / 3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


Train on 23730 samples, validate on 11865 samples
Epoch 1/3
23730/23730 [==============================] - 111s 5ms/step - loss: 1.1403 - acc: 0.5853 - val_loss: 0.1108 - val_acc: 0.9641
Epoch 2/3
23730/23730 [==============================] - 108s 5ms/step - loss: 0.1039 - acc: 0.9672 - val_loss: 0.0246 - val_acc: 0.9919
Epoch 3/3
23730/23730 [==============================] - 108s 5ms/step - loss: 0.0397 - acc: 0.9877 - val_loss: 0.0102 - val_acc: 0.9965
Running Fold 2 / 3
Train on 23730 samples, validate on 11865 samples
Epoch 1/3
23730/23730 [==============================] - 108s 5ms/step - loss: 1.0151 - acc: 0.6399 - val_loss: 0.0911 - val_acc: 0.9714
Epoch 2/3
23730/23730 [==============================] - 107s 5ms/step - loss: 0.0880 - acc: 0.9721 - val_loss: 0.0214 - val_acc: 0.9944
Epoch 3/3
23730/23730 [==============================] - 107s 5ms/step - loss: 0.0334 - acc: 0.9897 - val_loss: 0.0091 - val_acc: 0.9973
Running Fold 3 / 3
Train on 23730 samples, validate on 1186

### Train model on all training data

In [0]:
if not train_model_using_k_fold:
    model = make_model(img_rows, img_cols, nb_classes)
    results = train_and_evaluate_model(
        model,
        data_train,
        labels_train,
        data_test,
        labels_test,
        target_names,
        nb_classes)

## Analysis and Saving of Results

In [32]:
if train_model_using_k_fold: 
    for fold, result in results.items():
        print("Fold: ", fold)
        print(result["classification_report"])
        print("- - - - - - - - - - - - - - - - - - - -")

Fold:  1
                  precision    recall  f1-score   support

ArmFasterTowards       0.99      1.00      1.00      1695
ArmSlowerTowards       1.00      1.00      1.00      1695
     CirclingArm       1.00      0.99      0.99      1695
        Clapping       1.00      1.00      1.00      1695
       PickingUp       1.00      0.99      0.99      1695
         Sitting       0.99      1.00      0.99      1695
         Walking       1.00      1.00      1.00      1695

     avg / total       1.00      1.00      1.00     11865

- - - - - - - - - - - - - - - - - - - -
Fold:  2
                  precision    recall  f1-score   support

ArmFasterTowards       1.00      1.00      1.00      1695
ArmSlowerTowards       1.00      1.00      1.00      1695
     CirclingArm       1.00      1.00      1.00      1695
        Clapping       1.00      1.00      1.00      1695
       PickingUp       0.99      0.99      0.99      1695
         Sitting       1.00      0.99      1.00      1695
         W

In [0]:
if not train_model_using_k_fold: 
    print(results["classification_report"])

### Plot and Save graph

In [15]:
if not load_models:
    visualize.plot_train_test_acc(history)


NameError: ignored

### Save Models

In [0]:
if not load_models and save_models:
    model.save(MODEL_PATH + name + '.h5')

### Save Classification Report and Confusion Matricies

In [0]:
if save_reports:
    file = open(REPORT_PATH + 'Classification_and_Confusion.txt', 'w') 

    file.write("--------------------------------------------------\n") 
    file.write("Model Name: " + name + "\n") 
    file.write("Classification Report:\n") 
    file.write(classification_rep)
    file.write("Confusion Matrix:\n") 
    file.write(np.array2string(confusion) + "\n")

    file.close() 